# 基於內容的推薦方法 (Content based)

實驗利用基金屬性資料的推薦法

## 概念: 

* 蒐集基金六種屬性
        1. 投資區域
        2. 基金類型
        3. 配息頻率
        4. 基金目前規模區間
        5. 高收益債
        6. 風險屬性
        
* 利用已買的基金屬性, 推算用戶**偏好**屬性**權重**
    
    
    1. 已買 J12 (新興市場, 組合型, 10-100億元, 高風險), J23 (新興市場,平衡型, 100億元以上,高風險) ==> 特徵: 新興市場(2/2), 高風險(2/2), 組合型(1/2),平衡型(1/2),10-100億元(1/2),100億元(1/2)
    
    2. 候選基金中,計算分數 新興市場(2/2) + 高風險(2/2) + 組合型(1/2) + 平衡型(1/2) + 10-100億元(1/2) + 100億元以上(1/2) 
    
    3. 排序

In [1]:
import pandas as pd
import pypyodbc 
from tqdm import tqdm 
import numpy as np 
import pickle

In [2]:
conn = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=test")

## 整理基金屬性

In [3]:
df_item_features = pd.read_sql("""
    select [基金代碼],
        [投資區域],
        [基金類型],
        [配息頻率],
        [基金目前規模區間],
        [高收益債],
        [風險屬性]
    from ihong_基金推薦demo_基金特徵2
""",conn)
df_item_features.head(3)

,基金代碼,投資區域,基金類型,配息頻率,基金目前規模區間,高收益債,風險屬性
0,100,台灣,股票型,None,d.10~100億元台幣,None,None
1,101,台灣,股票型,None,c.5~10億元台幣,None,None
2,103,台灣,股票型,None,c.5~10億元台幣,None,None


## 用戶購買資料

In [4]:
with open('./funds/sp_funds_datasets.pickle','rb') as f:
    data = pickle.load(f)

In [5]:
test = data['test']
train = data['train']
user_idxs = data['user_idxs']
idx_to_userid = data['idx_to_userid']
userid_to_idx = data['userid_to_idx']
idx_to_itemid = data['idx_to_itemid']
itemid_to_idx = data['itemid_to_idx']

fundid_names_df = pd.read_csv('./funds/fundid_to_name.csv',encoding='cp950')
fundid_to_names = {}

for d in fundid_names_df.to_dict('records'):
    fundid_to_names[d['基金代碼']] = d['基金中文名稱']

In [6]:
df_item_features['iidx'] = df_item_features['基金代碼'].\
    apply(lambda row,mapper:mapper.get(row,np.nan),args=[itemid_to_idx])
df_item_features.head(3)    

,基金代碼,投資區域,基金類型,配息頻率,基金目前規模區間,高收益債,風險屬性,iidx
0,100,台灣,股票型,None,d.10~100億元台幣,None,None,871.0
1,101,台灣,股票型,None,c.5~10億元台幣,None,None,1355.0
2,103,台灣,股票型,None,c.5~10億元台幣,None,None,198.0


### 測試單一用戶購買的基金特徵    

In [7]:
test_uidx = 100
user_pur_item_indx = train[test_uidx,].indices
test_item_df = df_item_features[df_item_features['iidx'].isin(user_pur_item_indx)]
test_item_df

,基金代碼,投資區域,基金類型,配息頻率,基金目前規模區間,高收益債,風險屬性,iidx
422,63Z,中國,股票型,None,h.1~5兆元台幣,None,高風險,32.0
627,770,全球,股票型,None,h.1~5兆元台幣,None,None,75.0
664,78M,歐洲,股票型,None,h.1~5兆元台幣,None,None,61.0
667,78S,歐洲,股票型,None,h.1~5兆元台幣,None,None,14.0
754,83F,新興市場,股票型,None,h.1~5兆元台幣,None,高風險,1938.0
1541,MB4,全球,股票型,None,i.5~10兆元台幣,None,None,24.0
1736,T37,全球,平衡型,月配,i.>10兆元台幣,高收益債,None,162.0
1911,Y38,美國,股票型,None,i.5~10兆元台幣,None,None,107.0


### 彙整成`dict`

In [8]:
from collections import defaultdict

In [9]:
test_item_df.shape[0]

8

In [10]:
# defaultdict
tmp = defaultdict(int)
num_of_pur_by_user = test_item_df.shape[0]
for idx, row in test_item_df.iterrows():
    for e in row[1:-1].tolist():
        if e:
            tmp[e]+=1
total = sum(tmp.values())
tmp2 = {k:v/ num_of_pur_by_user for k,v in tmp.items()}
print('----'*10)
for k in sorted(tmp2,key=tmp2.get,reverse=True)[:20]:
    print('特徵:{}'.format(k))
    print('權重:{:.2f}'.format(tmp2[k]))
    print('----'*10)

----------------------------------------
特徵:股票型
權重:0.88
----------------------------------------
特徵:h.1~5兆元台幣
權重:0.62
----------------------------------------
特徵:全球
權重:0.38
----------------------------------------
特徵:高風險
權重:0.25
----------------------------------------
特徵:歐洲
權重:0.25
----------------------------------------
特徵:i.5~10兆元台幣
權重:0.25
----------------------------------------
特徵:中國
權重:0.12
----------------------------------------
特徵:新興市場
權重:0.12
----------------------------------------
特徵:平衡型
權重:0.12
----------------------------------------
特徵:月配
權重:0.12
----------------------------------------
特徵:i.>10兆元台幣
權重:0.12
----------------------------------------
特徵:高收益債
權重:0.12
----------------------------------------
特徵:美國
權重:0.12
----------------------------------------


### 所有用戶的購買特徵

In [11]:
user_numbers = train.shape[0]
users_pur_profile = {}
for uidx in tqdm(range(user_numbers)):
    user_pur_item_idxs = train[uidx,].indices
    user_pur_items_df = df_item_features[df_item_features['iidx'].isin(user_pur_item_idxs)]
    
    user_profile = defaultdict(int)
    num_of_pur_by_user = user_pur_items_df.shape[0]
    for idx, row in user_pur_items_df.iterrows():
        for e in row[1:-1].tolist():
            if e:
                user_profile[e] +=1
#     total = sum(user_profile.values())
    
    user_profile = {k:v/ num_of_pur_by_user for k,v in user_profile.items()}
    
    users_pur_profile[uidx] = user_profile

100%|███████████████████████████████████| 26324/26324 [00:36<00:00, 714.83it/s]


In [12]:
test_uidx = 100
for w in sorted(users_pur_profile[test_uidx],key=users_pur_profile[test_uidx].get, reverse=True):
    print('特徵:{},\t權重:{:.2f}'.format(w,users_pur_profile[test_uidx][w]))
    print('--'*20)

特徵:股票型,	權重:0.88
----------------------------------------
特徵:h.1~5兆元台幣,	權重:0.62
----------------------------------------
特徵:全球,	權重:0.38
----------------------------------------
特徵:高風險,	權重:0.25
----------------------------------------
特徵:歐洲,	權重:0.25
----------------------------------------
特徵:i.5~10兆元台幣,	權重:0.25
----------------------------------------
特徵:中國,	權重:0.12
----------------------------------------
特徵:新興市場,	權重:0.12
----------------------------------------
特徵:平衡型,	權重:0.12
----------------------------------------
特徵:月配,	權重:0.12
----------------------------------------
特徵:i.>10兆元台幣,	權重:0.12
----------------------------------------
特徵:高收益債,	權重:0.12
----------------------------------------
特徵:美國,	權重:0.12
----------------------------------------


## 基於每個用戶的選購特徵作內容推薦

In [13]:
temp = next(df_item_features.iterrows())[1]
temp

基金代碼                 100
投資區域                  台灣
基金類型                 股票型
配息頻率                None
基金目前規模區間    d.10~100億元台幣
高收益債                None
風險屬性                None
iidx                 871
Name: 0, dtype: object

In [79]:
len(users_pur_profile.keys())

26324

In [14]:
def get_cb_scores(uidx,df_item_features,user_pur_profile):
    """for a uidx calculate content based scores w.r.t user_pur_profile """
    scores = {}
    for _, row in df_item_features.iterrows():
        score = 0
        iidx = row['iidx']
        for key in row[1:-1]:
            if user_pur_profile.get(key):
                score += user_pur_profile.get(key)
        scores[iidx] = score
    return scores
        

In [15]:
test_uidx = 100
uidx100_cb_scores = get_cb_scores(test_uidx,df_item_features,users_pur_profile[test_uidx])


### 觀察其中一個用戶的推薦結果

In [16]:
set(df_item_features[df_item_features['基金代碼'] =='IQ7'].loc[:,'投資區域':'風險屬性'].values.flatten())

{'全球', None, '高風險', '股票型', 'h.1~5兆元台幣'}

In [17]:
test_uidx = 100
for w in sorted(users_pur_profile[test_uidx],key=users_pur_profile[test_uidx].get, reverse=True):
    print('特徵:{},\t權重:{:.2f}'.format(w,users_pur_profile[test_uidx][w]))
    print('--'*20)

uidx100_cb_scores = get_cb_scores(test_uidx,df_item_features,users_pur_profile[test_uidx])

for iidx in sorted(uidx100_cb_scores,key=uidx100_cb_scores.get,reverse=True)[:10]: ## top 20 iidx
    print('基金代碼:{}'.format(idx_to_itemid.get(iidx)))
    fund_feat_set = set(df_item_features[df_item_features['iidx'] ==iidx].loc[:,'投資區域':'風險屬性'].values.flatten())
    user_feat_set = set(users_pur_profile[test_uidx].keys())
    union_feat_set = fund_feat_set.intersection(user_feat_set)
    print('\t 共同特徵:{}'.format(union_feat_set))

特徵:股票型,	權重:0.88
----------------------------------------
特徵:h.1~5兆元台幣,	權重:0.62
----------------------------------------
特徵:全球,	權重:0.38
----------------------------------------
特徵:高風險,	權重:0.25
----------------------------------------
特徵:歐洲,	權重:0.25
----------------------------------------
特徵:i.5~10兆元台幣,	權重:0.25
----------------------------------------
特徵:中國,	權重:0.12
----------------------------------------
特徵:新興市場,	權重:0.12
----------------------------------------
特徵:平衡型,	權重:0.12
----------------------------------------
特徵:月配,	權重:0.12
----------------------------------------
特徵:i.>10兆元台幣,	權重:0.12
----------------------------------------
特徵:高收益債,	權重:0.12
----------------------------------------
特徵:美國,	權重:0.12
----------------------------------------
基金代碼:IO6
	 共同特徵:{'高風險', '股票型', 'h.1~5兆元台幣', '全球'}
基金代碼:IQ7
	 共同特徵:{'高風險', '股票型', 'h.1~5兆元台幣', '全球'}
基金代碼:X01
	 共同特徵:{'高風險', '股票型', 'h.1~5兆元台幣', '全球'}
基金代碼:52L
	 共同特徵:{'股票型', '月配', 'h.1~5兆元台幣', '全球'}
基金代碼:72A
	 共同特徵:{'股票型', '月配', 'h.1~5兆元台幣', '

## 評估內容推薦結果

In [21]:
## 在train set 裡面給出10個推薦基金, 評估recall 
def get_cb_rec_items(uidx,users_pur_profile,df_item_features,topN = 10):
    """
    
    params
    ======
    uidx : (int) user index 
    users_pur_profile : (dict) features scores for every useridx 
                        ex: {
                                uidx1: {
                                        feat1:score1,
                                        feat2:score2,...
                                    },
                                uidx0:{
                                        feat1:score1,
                                        feat2:score2,...
                                }
                            }
    df_item_features : (dataframe) all items features 
    topN : (int) numbers of recommended items
    
    return
    ======
    
    (DataFrame) recommended topN funds, scores, reason(union features)
    """
    uidx_cb_scores = get_cb_scores(uidx,df_item_features,users_pur_profile[test_uidx])
    
#     fundid_rec_topN = [idx_to_itemid.get(iidx) for iidx in 
#                        sorted(uidx_cb_scores,key=uidx100_cb_scores.get,reverse=True)[:topN]] ## topN iidx
#     scores = [uidx_cb_scores.get(fundid) for fundid in fundid_rec_topN]
    
    fundid_rec_topN = []
    scores = []
    union_feat_sets = []
    reasons = []
    
    for iidx in sorted(uidx_cb_scores,key=uidx_cb_scores.get,reverse=True)[:topN]:        
        fundid = idx_to_itemid.get(iidx)
        score = uidx_cb_scores.get(iidx)
        
        fund_feat_set = set(df_item_features[df_item_features['iidx'] ==iidx].loc[:,'投資區域':'風險屬性'].values.flatten())
        user_feat_set = set(users_pur_profile[uidx].keys())
        union_feat_set = fund_feat_set.intersection(user_feat_set)
        
        reason_list = [str(users_pur_profile[uidx][list(union_feat_set)[ii]])[:5] + '*' + list(union_feat_set)[ii] \
                       for ii in range(len(union_feat_set))]
        reason_str = ' + '.join(reason_list)
        reasons.append(reason_str)
        
        fundid_rec_topN.append(fundid)
        scores.append(score)
        union_feat_sets.append(union_feat_set)                
        
    return pd.DataFrame({'fundid':fundid_rec_topN, 'score':scores, 'tag_features':reasons})

In [57]:
str(users_pur_profile[100][list(union_feat_set)[3]]) + '*' + list(union_feat_set)[3] 

'0.375*全球'

In [67]:
str(users_pur_profile[100][list(union_feat_set)[3]])[:5] + '*' + list(union_feat_set)[3] 

'0.375*全球'

In [22]:
get_cb_rec_items(100,users_pur_profile,df_item_features)

,fundid,score,tag_features
0,IO6,2.125,0.25*高風險 + 0.875*股票型 + 0.625*h.1~5兆元台幣 + 0.375*全球
1,IQ7,2.125,0.25*高風險 + 0.875*股票型 + 0.625*h.1~5兆元台幣 + 0.375*全球
2,X01,2.125,0.25*高風險 + 0.875*股票型 + 0.625*h.1~5兆元台幣 + 0.375*全球
3,52L,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
4,72A,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
5,72B,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
6,74Z,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
7,75R,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
8,75S,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球
9,77P,2.000,0.875*股票型 + 0.125*月配 + 0.625*h.1~5兆元台幣 + 0.375*全球


In [45]:
uidx_test,iidx_test = test.nonzero()

In [46]:
rec_fundids_given_uidx = get_cb_rec_items(100,users_pur_profile,df_item_features,10)

In [219]:
## calculate recall ##

hit = 0 
for index, uidx in tqdm(enumerate(uidx_test)):
    rec_fundids = get_cb_rec_items(uidx,users_pur_profile,df_item_features,10)['fundid']
    if iidx_test[index] in rec_fundids:
        hit += 1
print('hit:{}'.format(hit))
print('recall:{:.2f} %'.format(100*hit/len(uidx_test)))

5264it [45:59,  1.91it/s]


hit:553
recall:0.10505319148936171


In [221]:
print('recall:{:.2f}%'.format(100*hit/len(uidx_test)))

recall:10.51%


## 推薦基金結果 - 內容推薦清單

In [23]:
rec_temp = get_cb_rec_items(1,users_pur_profile,df_item_features,10)
rec_temp['model'] = 'content_based'
rec_temp['rank'] = rec_temp.index + 1

In [46]:
rec_temp1 = get_cb_rec_items(20,users_pur_profile,df_item_features,10)
rec_temp1['userid'] = idx_to_userid[20]
rec_temp1['model'] = 'content_based'
rec_temp1['rank'] = rec_temp1.index + 1

In [47]:
rec_temp1

,fundid,score,tag_features,userid,model,rank
0,IO6,2.125,0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.35...,A1597233410,content_based,1
1,IQ7,2.125,0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.35...,A1597233410,content_based,2
2,X01,2.125,0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.35...,A1597233410,content_based,3
3,52L,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,4
4,72A,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,5
5,72B,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,6
6,74Z,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,7
7,75R,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,8
8,75S,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,9
9,77P,2.000,0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球,A1597233410,content_based,10


In [49]:
for row in rec_temp1.iterrows():
    print(row[1].to_dict())

{'fundid': 'IO6', 'score': 2.125, 'tag_features': '0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 1}
{'fundid': 'IQ7', 'score': 2.125, 'tag_features': '0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 2}
{'fundid': 'X01', 'score': 2.125, 'tag_features': '0.235*高風險 + 0.176*股票型 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 3}
{'fundid': '52L', 'score': 2.0, 'tag_features': '0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 4}
{'fundid': '72A', 'score': 2.0, 'tag_features': '0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 5}
{'fundid': '72B', 'score': 2.0, 'tag_features': '0.176*股票型 + 0.470*月配 + 0.176*h.1~5兆元台幣 + 0.352*全球', 'userid': 'A1597233410', 'model': 'content_based', 'rank': 6}
{'fundid': '7

In [50]:
# cursor = conn.cursor()
insert_sql = "INSERT INTO {0}({1}) VALUES ({2})"
tablename = 'ihong_基金推薦demo_推薦清單'
num_quest = '?'+ ',?'*(len(row[1])-1)
fields = ','.join(row[1].to_dict().keys())
values = list(row[1].to_dict().values())
print(insert_sql.format(tablename,fields,num_quest))
cursor.execute(insert_sql.format(tablename, fields,num_quest), values)

INSERT INTO ihong_基金推薦demo_推薦清單(fundid,score,tag_features,userid,model,rank) VALUES (?,?,?,?,?,?)


In [62]:
## build data frame ##
insert_sql = "INSERT INTO {0}({1}) VALUES ({2})"
tablename = 'ihong_基金推薦demo_推薦清單'
num_quest = '?'+ ',?'*(5)

cursor = conn.cursor()

for uidx in tqdm(range(train.shape[0])):
    ## 結果(全清單)塞入資料庫
    rec_fundids = get_cb_rec_items(uidx,users_pur_profile,df_item_features,10)
    rec_fundids['userid'] = idx_to_userid[uidx]
    rec_fundids['model'] = 'content_based'
    rec_fundids['rank'] = rec_temp1.index + 1
    
    for row in rec_fundids.iterrows():
        fields = ','.join(row[1].to_dict().keys())
        values = list(row[1].to_dict().values())
        cursor.execute(insert_sql.format(tablename, fields,num_quest), values)
#     print('uidx:{} inserted'.format(uidx))
cursor.commit()


100%|██████████████████████████████████| 26324/26324 [2:36:33<00:00,  2.87it/s]
